# FIT5196 Task 1 in Assessment 2
#### Student Name: Boyu Zhang
#### Student ID: 28491300

Date: XX/XX/XXXX

Version: 1.0

Environment: Python 3.6 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* pandas (for dataframe, included in Anaconda Python ) 
* re (for regular expression, included in Anaconda Python 3.6) 
* numpy (for numpy array, included in Anaconda Python 3.6) 



## 1.  Import libraries 

In [1]:
# Code to import libraries as you need in this assessment, e.g.,
import pandas as pd
import numpy as np
import re

## 2. Load data and exploration


In [2]:
df1 = pd.read_csv('./Input/dataset1_with_error.csv')

In [3]:
#overview of the dataset
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25077 entries, 0 to 25076
Data columns (total 11 columns):
Id                  25077 non-null int64
Title               25077 non-null object
Location            25077 non-null object
ContractType        25077 non-null object
ContractTime        25077 non-null object
Company             21242 non-null object
Category            25077 non-null object
Salary per annum    25077 non-null object
SourceName          25077 non-null object
OpenDate            25077 non-null object
CloseDate           25077 non-null object
dtypes: int64(1), object(10)
memory usage: 2.1+ MB


In [4]:
df1.describe(include=['O'])

,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
count,25077,25077,25077,25077,21242,25077,25077,25077,25077,25077
unique,25077,482,3,3,4879,8,1589,90,2203,2400
top,Mechatronics Design Engineer,UK,not available,permanent,UKStaffsearch,IT Jobs,35004,totaljobs.com,20130127T150000,20130526T120000
freq,1,3996,19499,16194,248,7085,1011,5335,27,23


In [5]:
#number of missing values in each column
df1.isnull().sum()

Id                     0
Title                  0
Location               0
ContractType           0
ContractTime           0
Company             3835
Category               0
Salary per annum       0
SourceName             0
OpenDate               0
CloseDate              0
dtype: int64

### Highlight of initial investigation
- There are 9 columns of categorical values, and 2 column of numeric values -- `Id` and `Salary`
- There are a lot of missing values in the `Company` column
- There must be issue with the column `Title` since the most frequent value is like "** *Ton**"which is obviously invalid
- The most frequent value of `Location` is "UK" which suspicious
- There are potential semantic anomalies :
    - unique values can have redundance
    - The violation of two Date
- There can be sytax anomalies:
     - inconistent representation of values in the same column
     - invalid value of categorical values

In [6]:
#first glance of the dataset
df1.head()

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
0,12612628,Engineering Systems Analyst,Dorking,not available,permanent,Gregory Martin International,Engineering Jobs,24996,cv-library.co.uk,20121103T000000,20121203T000000
1,12612830,Stress Engineer Glasgow,Glasgow,not available,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,20130108T150000,20130408T150000
2,12612844,Modelling and simulation analyst,Hampshire,not available,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,20130726T150000,20130924T150000
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Surrey,not available,permanent,Gregory Martin International,Engineering Jobs,27504,cv-library.co.uk,20121214T000000,20130314T000000
4,12613647,"Pioneer, Miser Engineering Systems Analyst",Surrey,not available,permanent,Gregory Martin International,Engineering Jobs,24996,cv-library.co.uk,20131025T000000,20131224T000000


In [7]:
df1.iloc[3].Title

'Engineering Systems Analyst / Mathematical Modeller'

## 3. Identify issues and fix 

### ID
For the column `ID` we don't have any constraints other than non-redundance. Thus we only check whether all **IDs** is unique.

In [8]:
#test if all Id 
df1.Id.unique().shape[0] == df1.shape[0]

True

### Title
- It's reasonable no matter there are redundant tiltles or not.
- The values should be in valid language

In [9]:
#test whethere there is redundant titles
df1.Title.unique().shape

(25077,)

In [10]:
df1.Title.value_counts()

Mechatronics Design Engineer                                             1
Project Manager  St Albans to ****k                                      1
Photocopier / Printer Engineer  Southampton                              1
IT Lecturer                                                              1
Deputy Nursing Home Manager  Maidstone                                   1
Chef de Partie, Central London                                           1
Telemarketing Supervisor Hampshire                                       1
Health Care Assistant – HCA  West Midlands  Birmingham                   1
Recovery Worker                                                          1
RGN Clinical Lead Nurse Warminster                                       1
Graduate Sales Executive  IT industry                                    1
C Developer / Web Developer / NET Developer – West London                1
IDTS Nurse                                                               1
Junior Flash Developer  L

- It appears there are values that contain invalid characters in the `Title` column 
- It seems some titles also contains the company name or location which could be improved for clearity of data

In [11]:
df1[df1.Title=='**** Ton']

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
24644,69193100,**** Ton,Rugby,full_time,permanent,Interaction Recruitment plc,Engineering Jobs,13440,fish4.co.uk,20130815T120000,20131113T120000


In [12]:
df1[df1.Title=='Electronics Engineer, Embedded Software, R D, York, ****']

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
2987,65102296,"Electronics Engineer, Embedded Software, R D, ...",North Yorkshire,full_time,permanent,Amrec,Engineering Jobs,37500,thecareerengineer.com,20130926T120000,20131225T120000


In [13]:
df1[df1.Title=='Assistant Manager  South West London Gastro Pub  ****K']

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
18230,68706648,Assistant Manager South West London Gastro Pu...,London,not available,not available,Cherryred Recruitment,Hospitality & Catering Jobs,22500,jobs.catererandhotelkeeper.com,20130702T000000,20130831T000000


#### Temporary conclusions:
- After inspecting some records that contains invalid character, there seems no clues how to replace with meaningful values. Thus we think it is better to keep these invalid character, since R****** still tell the length of the word but 'R' will not

### Location 


In [14]:
df1[df1.Location == 'UK']

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
9,22933091,Chef de Partie Award Winning Dining Live In ...,UK,not available,not available,Chef Results,Hospitality & Catering Jobs,18000,caterer.com,20120814T000000,20120828T000000
12,23530231,Lead Engineers (Stress),UK,not available,permanent,Gregory Martin International,Engineering Jobs,39504,cv-library.co.uk,20121113T120000,20130211T120000
13,24104334,"Relief Chef de Partie Croydon, Surrey Live in",UK,not available,not available,Towngate Personnel,Hospitality & Catering Jobs,17280,caterer.com,20131029T000000,20131128T000000
26,27739522,Welwyn Chef de Partie does it get any better t...,UK,not available,not available,Clear Selection,Hospitality & Catering Jobs,18000,caterer.com,20130106T120000,20130307T120000
27,27740574,Pastry Chef AL**** ****AA Rosette Restaurant,UK,not available,not available,Clear Selection,Hospitality & Catering Jobs,18996,caterer.com,20120320T120000,20120403T120000
30,29650985,Fine Dining Chefs for rosette restaurants up ...,UK,not available,not available,Clear Selection,Hospitality & Catering Jobs,24000,caterer.com,20131205T150000,20140104T150000
31,30192171,Web Developer,UK,not available,permanent,Indigo 21 Ltd,IT Jobs,35004,cv-library.co.uk,20130305T000000,20130504T000000
39,31989124,Mechanical Engineer Design and Substantiation,UK,not available,permanent,Gregory Martin International,Engineering Jobs,35004,cv-library.co.uk,20131018T120000,20131117T120000
42,32233421,Principle Mechanical Engineer,UK,not available,permanent,Rullion Engineering Ltd,Engineering Jobs,48504,cv-library.co.uk,20130816T120000,20131114T120000
43,32417995,Chef de Partie Award Winning Fine Dining Rest...,UK,not available,not available,Chef Results,Hospitality & Catering Jobs,20496,caterer.com,20131016T150000,20131030T150000


Problem of location seems more complicated:
 - some records has just no detailed `Location` but just "UK"(default filling for none)
 - some has the location data but it is included in the `Title` value and these record also just have the value of `Location` as "UK"

#### Solution:
- For the one with no specigic `Location` we cannot tell where to get this `Location` information

### ContractType

In [16]:
#check unique values of the columns
df1.ContractType.unique()

array(['not available', 'full_time', 'part_time'], dtype=object)

According to dta dataset description, the 'not available' should be replaced by 'non_specified'

In [17]:
df1.ContractType.replace('not available','non_specified',inplace=True)

### ContractTime

In [18]:
#check unique values
df1.ContractTime.unique()

array(['permanent', 'not available', 'contract'], dtype=object)

In [19]:
df1.ContractTime.replace('not available','non_specified',inplace=True)

### Company

In [20]:
df1.Company.value_counts()

UKStaffsearch                              248
CVbrowser                                  170
Randstad                                   169
JOBG8                                      159
Matchtech Group plc.                       150
Chef Results                               137
Penguin Recruitment                        134
Hays                                       115
London4Jobs                                112
COREcruitment International                111
JAM Recruitment Ltd                        109
Computer People                             99
Clear Selection                             92
Monarch Recruitment                         89
Towngate Personnel                          79
Cherryred Recruitment                       69
Senitor Associates                          67
Modis                                       62
Capita Resourcing                           62
Support Services Group                      62
Aspire Data Recruitment                     61
Adecco       

In [21]:
df1.Company.str.contains('LTD').sum(),df1.Company.str.contains('Limited').sum(),df1.Company.str.contains('Ltd').sum()

(153, 1251, 3604)

- Seems there are some minor irregularities like the inconsitent use of "Limited","Ltd" and "LTD", which can all be unified as "Ltd"
- To be consistent "plc" can be converted into "Plc"

In [30]:
# unify all post-fix of company name
df1.Company = df1.Company.str.replace('plc','Plc')
df1.Company = df1.Company.str.replace('Limited','Ltd')
df1.Company = df1.Company.str.replace('LTD','Ltd')

### Company
It is already discovered before that there is large amount of missing values in the `Company` column

In [ ]:
df1.Company.isna().sum()

It seems that it is impossible to fill in the name of these companies, but depends on whether it was just a system error or typo, and also regarding different usage of the dataset, it is better we keep this records.

### Category

In [ ]:
df1.Category.value_counts()

- No invalid values and no redundance
- Also no inconsistent represnetation of same value

### Salary

In [ ]:
df1["Salary per annum"].unique()

There are inconsiten representations of the salary

In [ ]:
p1 = '^\d+$'

In [ ]:
non_numeric_salary = df1['Salary per annum'].apply(lambda x: False if re.match(p1,x) else True )

In [ ]:
inconsis_salary =df1['Salary per annum'][non_numeric_salary];inconsis_salary.value_counts()

There seems to have two type of invalid representations:
1. "90K"
2. "21819.6 - 24116.4"

**For the first one we just transfer the str representation to int.**

**For the second one we pick the mean of the range and round them to the nearest integer.**

In [ ]:
#design the regex patterns
p1 = '(\d+)K'#regex for '90K'
p2 = '(\d+\.\d+)\s*-\s*(\d+\.\d+)'#regex for '3333.222 - 34444.333'
p3 = '^\d+$' # regex for normal representation

In [ ]:
#the method for unifying the values of salaries
def unify_salary(sp):
    r1 = re.match(p1,sp)
    r2 = re.match(p2,sp)
    if r1:
        return r1.group(1) + '000'
    if r2:
        return float(r2.group(1))+float(r2.group(2))/2
    if re.match(p3,sp):
        return sp

In [ ]:
df1['Salary per annum'] = df1['Salary per annum'].apply(unify_salary)

In [ ]:
#for space saving, salary won't be that large, so transfer to int32 is enough
df1['Salary per annum'] = df1['Salary per annum'].astype(int);

Then we need to check if there are semantic errors in the salary column:
- one error can be the occurence of error or negative number

In [ ]:
df1.shape[0] - (df1['Salary per annum'] <= 0).sum() == df1.shape[0]

Thia proves there is not zero or negative values in the row of `Salary per annum`

### SourceName

In [ ]:
df1.SourceName.value_counts()

Seems no explicit issues.

### OpenData and CloseDate
We assume at the first place that the date form is Y-M-D

In [ ]:
df1.OpenDate.head()

The result of above cell proves our assumption, so we can start invetigate whether there are issues in the two columns for date.

In [ ]:
p = '\d{4}(\d{2})(\d{2}).(\d{2})(\d{2})(\d{2})'
def is_valid_date(d):
    month_invalid = int(re.match(p,d).group(1)) > 12 
    day_invalid = int(re.match(p,d).group(2)) > 31 #keep it simple fallacious for the moment, for simplicity just assume each month has 31 days
    hr_invalid = int(re.match(p,d).group(3)) > 23
    min_invalid = int(re.match(p,d).group(4)) > 59
    second_invalid = int(re.match(p,d).group(5)) > 59
    if month_invalid or day_invalid or hr_invalid or min_invalid or second_invalid:
        return d
    else:
        return np.nan
    
invalid_open_date = df1.OpenDate.apply(is_valid_date)
invalid_close_date = df1.CloseDate.apply(is_valid_date)

In [ ]:
#test
print(invalid_open_date.dropna().shape, invalid_close_date.dropna().shape)

indicates there are 10 invalid date 

inspect the invalid values

In [ ]:
invalid_series = invalid_open_date.dropna().values

In [ ]:
df1[df1.OpenDate.isin(invalid_series)]

It seems though these OpenDates are invalid, but if the month and day get reversed they are all valid date.

Thus we decide to reverse them.

In [ ]:
df1.loc[df1.OpenDate.isin(invalid_series), 'OpenDate'] = df1[df1.OpenDate.isin(invalid_series)]['OpenDate'].apply(lambda s: s[:4] + s[6:8] + s[4:6] + s[8:])

In [ ]:
#transform the string representation of date to type datetime for more flexibility
df1[['OpenDate','CloseDate']] = df1[['OpenDate','CloseDate']].apply(lambda x: pd.to_datetime(x))

Then we need to check semantic errors, which is that that the OpenDate later than the CloseDate

In [ ]:
df1[df1.OpenDate > df1.CloseDate]

We propose a solution to this semantic error which is reverse the `OpenDate` and the `CloseDate`

In [ ]:
df1_date = df1[['OpenDate','CloseDate']]

In [ ]:
df1[['OpenDate','CloseDate']] = pd.concat([df1_date.min(axis=1), df1_date.max(axis=1)], axis=1)

## 3. Summary
Give a short summary of your work done above, such as your findings.